<a href="https://colab.research.google.com/github/alanwuha/ce7455-nlp/blob/master/assignment-1/word-level-language-model-rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Download wikitext-2 dataset

In [1]:
!rm -r data
!mkdir data
!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip -P /content/data
!unzip /content/data/wikitext-2-v1.zip -d /content/data

rm: cannot remove 'data': No such file or directory
--2020-02-06 09:58:33--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.168.133
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.168.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4475746 (4.3M) [application/zip]
Saving to: ‘/content/data/wikitext-2-v1.zip’

wikitext-2-v1.zip   100%[===================>]   4.27M  9.66MB/s    in 0.4s    

2020-02-06 09:58:34 (9.66 MB/s) - ‘/content/data/wikitext-2-v1.zip’ saved [4475746/4475746]

Archive:  /content/data/wikitext-2-v1.zip
   creating: /content/data/wikitext-2/
  inflating: /content/data/wikitext-2/wiki.test.tokens  
  inflating: /content/data/wikitext-2/wiki.valid.tokens  
  inflating: /content/data/wikitext-2/wiki.train.tokens  


### Specify arg variables

In [0]:
data = './data/wikitext-2'  # location of the data corpus
args_model = 'FFN'              # type of recurrent net
emsize = 200                # size of word embeddings
nhid = 200                  # number of hidden units per layer
nlayers = 2                 # number of layers
lr = 20                     # initial learning rate
clip = 0.25                 # gradient clipping
epochs = 40                 # upper epoch limit
batch_size = 20             # batch size
bptt = 35                   # sequence length
dropout = 0.2               # dropout applied to layers (0 = no dropout)
tied = False                 # tie the word embedding and softmax weights
seed = 1111                 # random seed
cuda = True                # use CUDA
log_interval = 200          # report interval
save = 'model.pt'           # path to save the final model
onnx_export = ''            # path to export the final model in onnx format
nhead = 2                   # the number of heads in the encoder/decoder of the transformer model

### data.py

In [0]:
import os
from io import open
import torch

class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(os.path.join(path, 'wiki.train.tokens'))
        self.valid = self.tokenize(os.path.join(path, 'wiki.valid.tokens'))
        self.test = self.tokenize(os.path.join(path, 'wiki.test.tokens'))

    def tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)

        # Add words to the dictionary
        with open(path, 'r', encoding="utf8") as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r', encoding="utf8") as f:
            idss = []
            for line in f:
                words = line.split() + ['<eos>']
                ids = []
                for word in words:
                    ids.append(self.dictionary.word2idx[word])
                idss.append(torch.tensor(ids).type(torch.int64))
            ids = torch.cat(idss)

        return ids

### model.py

In [0]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class FFNModel(nn.Module):
    """ Container module with an encoder, a feed forward module, and a decoder. """

    def __init__(self, ntokens, emsize, hidden_size, tie_weights=False):
        super(FFNModel, self).__init__()
        self.encoder = nn.Embedding(ntokens, emsize)
        self.hidden = nn.Linear(emsize, hidden_size)
        self.tanh = nn.Tanh()
        self.decoder = nn.Linear(hidden_size, ntokens)

        if tie_weights:
            if hidden_size != emsize:
                raise ValueError('When using the tied flag, hidden_size must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        # self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input):
        emb = self.encoder(input)
        hid = self.hidden(emb)
        output = self.tanh(hid)
        decoded = self.decoder(output)
        return decoded

class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5, tie_weights=False):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError( """An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""")
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        # Optionally tie weights as in:
        # "Using the Output Embedding to Improve Language Models" (Press & Wolf 2016)
        # https://arxiv.org/abs/1608.05859
        # and
        # "Tying Word Vectors and Word Classifiers: A Loss Framework for Language Modeling" (Inan et al. 2016)
        # https://arxiv.org/abs/1611.01462
        if tie_weights:
            if nhid != ninp:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output)
        return decoded, hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        if self.rnn_type == 'LSTM':
            return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                    weight.new_zeros(self.nlayers, bsz, self.nhid))
        else:
            return weight.new_zeros(self.nlayers, bsz, self.nhid)

# Temporarily leave PositionalEncoding module here. Will be moved somewhere else.
class PositionalEncoding(nn.Module):
    r"""Inject some information about the relative or absolute position of the tokens
        in the sequence. The positional encodings have the same dimension as
        the embeddings, so that the two can be summed. Here, we use sine and cosine
        functions of different frequencies.
    .. math::
        \text{PosEncoder}(pos, 2i) = sin(pos/10000^(2i/d_model))
        \text{PosEncoder}(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        \text{where pos is the word position and i is the embed idx)
    Args:
        d_model: the embed dim (required).
        dropout: the dropout value (default=0.1).
        max_len: the max. length of the incoming sequence (default=5000).
    Examples:
        >>> pos_encoder = PositionalEncoding(d_model)
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        r"""Inputs of forward function
        Args:
            x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        Examples:
            >>> output = pos_encoder(x)
        """

        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class TransformerModel(nn.Module):
    """Container module with an encoder, a recurrent or transformer module, and a decoder."""

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        try:
            from torch.nn import TransformerEncoder, TransformerEncoderLayer
        except:
            raise ImportError('TransformerEncoder module does not exist in PyTorch 1.1 or lower.')
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, has_mask=True):
        if has_mask:
            device = src.device
            if self.src_mask is None or self.src_mask.size(0) != len(src):
                mask = self._generate_square_subsequent_mask(len(src)).to(device)
                self.src_mask = mask
        else:
            self.src_mask = None

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return F.log_softmax(output, dim=-1)

### main.py

In [9]:
# coding: utf-8
import argparse
import time
import math
import os
import torch
import torch.nn as nn
import torch.onnx

# Set the random seed manually for reproducibility.
torch.manual_seed(seed)
if torch.cuda.is_available():
    if not cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

device = torch.device("cuda" if cuda else "cpu")

###############################################################################
# Load data
###############################################################################

# corpus = data.Corpus(data)
corpus = Corpus(data)

# Starting from sequential data, batchify arranges the dataset into columns.
# For instance, with the alphabet as the sequence and batch size 4, we'd get
# ┌ a g m s ┐
# │ b h n t │
# │ c i o u │
# │ d j p v │
# │ e k q w │
# └ f l r x ┘.
# These columns are treated as independent by the model, which means that the
# dependence of e. g. 'g' on 'f' can not be learned, but allows more efficient
# batch processing.

def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

eval_batch_size = 10
train_data = batchify(corpus.train, batch_size)
val_data = batchify(corpus.valid, eval_batch_size)
test_data = batchify(corpus.test, eval_batch_size)

###############################################################################
# Build the model
###############################################################################

ntokens = len(corpus.dictionary)
if args_model == 'Transformer':
    model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
elif args_model == 'FFN':
    model = FFNModel(ntokens, emsize, nhid, tied).to(device)
else:
    model = RNNModel(args_model, ntokens, emsize, nhid, nlayers, dropout, tied).to(device)

criterion = nn.CrossEntropyLoss()

###############################################################################
# Training code
###############################################################################

def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""

    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)


# get_batch subdivides the source data into chunks of length bptt.
# If source is equal to the example output of the batchify function, with
# a bptt-limit of 2, we'd get the following two Variables for i = 0:
# ┌ a g m s ┐ ┌ b h n t ┐
# └ b h n t ┘ └ c i o u ┘
# Note that despite the name of the function, the subdivison of data is not
# done along the batch dimension (i.e. dimension 1), since that was handled
# by the batchify function. The chunks are along dimension 0, corresponding
# to the seq_len dimension in the LSTM.

def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target


def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0.
    ntokens = len(corpus.dictionary)
    if args_model not in ['Transformer', 'FFN']:
        hidden = model.init_hidden(eval_batch_size)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            if args_model in ['Transformer', 'FFN']:
                output = model(data)
            else:
                output, hidden = model(data, hidden)
                hidden = repackage_hidden(hidden)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)


def train():
    # Turn on training mode which enables dropout.
    model.train()
    total_loss = 0.
    start_time = time.time()
    ntokens = len(corpus.dictionary)
    if args_model not in ['Transformer', 'FFN']:
        hidden = model.init_hidden(batch_size)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        model.zero_grad()
        if args_model in ['Transformer', 'FFN']:
            output = model(data)
        else:
            hidden = repackage_hidden(hidden)
            output, hidden = model(data, hidden)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // bptt, lr,
                elapsed * 1000 / log_interval, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()


def export_onnx(path, batch_size, seq_len):
    print('The model is also exported in ONNX format at {}'.
          format(os.path.realpath(onnx_export)))
    model.eval()
    dummy_input = torch.LongTensor(seq_len * batch_size).zero_().view(-1, batch_size).to(device)
    hidden = model.init_hidden(batch_size)
    torch.onnx.export(model, (dummy_input, hidden), path)


# Loop over epochs.
lr = lr
best_val_loss = None

# At any point you can hit Ctrl + C to break out of training early.
try:
    for epoch in range(1, epochs+1):
        epoch_start_time = time.time()
        train()
        val_loss = evaluate(val_data)
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
                'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                           val_loss, math.exp(val_loss)))
        print('-' * 89)
        # Save the model if the validation loss is the best we've seen so far.
        if not best_val_loss or val_loss < best_val_loss:
            with open(save, 'wb') as f:
                torch.save(model, f)
            best_val_loss = val_loss
        else:
            # Anneal the learning rate if no improvement has been seen in the validation dataset.
            lr /= 4.0
except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

# Load the best saved model.
with open(save, 'rb') as f:
    model = torch.load(f)
    # after load the rnn params are not a continuous chunk of memory
    # this makes them a continuous chunk, and will speed up forward pass
    # Currently, only rnn model supports flatten_parameters function.
    if model in ['RNN_TANH', 'RNN_RELU', 'LSTM', 'GRU']:
        model.rnn.flatten_parameters()

# Run on test data.
test_loss = evaluate(test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

if len(onnx_export) > 0:
    # Export the model in ONNX format.
    export_onnx(onnx_export, batch_size=1, seq_len=bptt)

| epoch   1 |   200/ 2983 batches | lr 20.00 | ms/batch  9.36 | loss  7.79 | ppl  2411.31
| epoch   1 |   400/ 2983 batches | lr 20.00 | ms/batch  9.26 | loss  7.29 | ppl  1469.03
| epoch   1 |   600/ 2983 batches | lr 20.00 | ms/batch  9.27 | loss  7.09 | ppl  1205.55
| epoch   1 |   800/ 2983 batches | lr 20.00 | ms/batch  9.29 | loss  6.93 | ppl  1021.64
| epoch   1 |  1000/ 2983 batches | lr 20.00 | ms/batch  9.26 | loss  6.76 | ppl   862.26
| epoch   1 |  1200/ 2983 batches | lr 20.00 | ms/batch  9.28 | loss  6.69 | ppl   801.76
| epoch   1 |  1400/ 2983 batches | lr 20.00 | ms/batch  9.30 | loss  6.66 | ppl   779.58
| epoch   1 |  1600/ 2983 batches | lr 20.00 | ms/batch  9.32 | loss  6.63 | ppl   756.83
| epoch   1 |  1800/ 2983 batches | lr 20.00 | ms/batch  9.28 | loss  6.51 | ppl   672.19
| epoch   1 |  2000/ 2983 batches | lr 20.00 | ms/batch  9.26 | loss  6.52 | ppl   679.17
| epoch   1 |  2200/ 2983 batches | lr 20.00 | ms/batch  9.24 | loss  6.43 | ppl   619.78
| epoch   

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type FFNModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


| epoch   2 |   200/ 2983 batches | lr 20.00 | ms/batch  9.33 | loss  6.34 | ppl   569.16
| epoch   2 |   400/ 2983 batches | lr 20.00 | ms/batch  9.39 | loss  6.32 | ppl   556.22
| epoch   2 |   600/ 2983 batches | lr 20.00 | ms/batch  9.31 | loss  6.18 | ppl   484.51
| epoch   2 |   800/ 2983 batches | lr 20.00 | ms/batch  9.29 | loss  6.20 | ppl   492.53
| epoch   2 |  1000/ 2983 batches | lr 20.00 | ms/batch  9.27 | loss  6.16 | ppl   472.83
| epoch   2 |  1200/ 2983 batches | lr 20.00 | ms/batch  9.27 | loss  6.19 | ppl   487.78
| epoch   2 |  1400/ 2983 batches | lr 20.00 | ms/batch  9.39 | loss  6.22 | ppl   502.30
| epoch   2 |  1600/ 2983 batches | lr 20.00 | ms/batch  9.26 | loss  6.24 | ppl   511.63
| epoch   2 |  1800/ 2983 batches | lr 20.00 | ms/batch  9.31 | loss  6.15 | ppl   466.95
| epoch   2 |  2000/ 2983 batches | lr 20.00 | ms/batch  9.44 | loss  6.17 | ppl   476.55
| epoch   2 |  2200/ 2983 batches | lr 20.00 | ms/batch  9.28 | loss  6.09 | ppl   440.49
| epoch   

### generate.py

In [0]:
data = './data/wikitext-2'  # location of the data corpus
checkpoint = './model.pt'   # model checkpoint to use
outf = 'generated.txt'      # output file for generated text
words = 1000                # number of words to generate
seed = 1111                 # random seed
cuda = True                 # use CUDA
temperature = 1.0           # temperature - higher will increase diversity
log_interval = 100          # reporting interval

In [20]:
###############################################################################
# Language Modeling on Wikitext-2
#
# This file generates new sentences sampled from the language model
#
###############################################################################

# Set the random seed manually for reproducibility.
torch.manual_seed(seed)
if torch.cuda.is_available():
  if not cuda:
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")

device = torch.device('cuda' if cuda else 'cpu')

# if temperature < 1e-3:
#   # parser.error('--temperature has to be greater or equal 1e-3')

with open(checkpoint, 'rb') as f:
  model = torch.load(f).to(device)
model.eval()

corpus = Corpus(data)
ntokens = len(corpus.dictionary)

is_transformer_model = hasattr(model, 'model_type') and model.model_type == 'Transformer'
if not is_transformer_model and args_model != 'FFN':
  hidden = model.init_hidden(1)
input = torch.randint(ntokens, (1, 1), dtype=torch.long).to(device)

with open(outf, 'w') as outf:
  with torch.no_grad(): # no tracking history
    for i in range(words):
      if is_transformer_model or args_model == 'FFN':
        output = model(input, False)
        word_weights = output[-1].squeeze().div(temperature).exp().cpu()
        word_idx = torch.multinomial(word_weights, 1)[0]
        word_tensor = torch.Tensor([[word_idx]]).long().to(device)
        input = torch.cat([input, word_tensor], 0)
      else:
        output, hidden = model(input, hidden)
        word_weights = output.squeeze().div(temperature).exp().cpu()
        word_idx = torch.multinomial(word_weights, 1)[0]
        input.fill_(word_idx)

      word = corpus.dictionary.idx2word[word_idx]

      outf.write(word + ('\n' if i % 20 == 19 else ' '))

      if i % log_interval == 0:
        print('| Generated {}/{} words'.format(i, words))

TypeError: ignored

In [0]:
!cat generated.txt

outside of Max and Chorus for purpose . Wippleman was the only Dutchman known in 1980 to explain his father
" Maroon 3 " . New of the Philadelphia Provident Paper entered the initial reflection of the book . "
<unk> " was moved to updating the peace format but compared his endanger of the reality sphere and Nahant to
communicate with many veterans of Marquis Turing 's virtual decision . <eos> Keith Lovell stated that early half of the
projects were the " most notable sexually <unk> as 1902 for the time . " These " actors carried agents
critics , both with a <unk> touched , organized production while Virgil , prone to marsh and green <unk> synths
, or a young and repeatedly thin @-@ digit object offered a beer curtain . " <eos> The track "
notorious " with a music game was serialized and a female editor he was from Los Angeles to sound from
studio art colors , from which the third Guy Fairies of the Association to @-@ East Division followed . In
2013 , Anderson invited to fund the Reich